In [26]:
import pandas as pd


data_frame = pd.read_csv("/content/Bead_prediction_data_6_features.csv")


#Preprocessing of data
feature_rename_dictionary = { 'Flory-Huggins χ parameter = (Mvol*Ra^2)/4RT':'Chi', 'Polymer Concentration (wt%)':"Conc.",
              'δ* =Sum of Products of Volume Fraction and δ of Individual Solvents':'delta',
              "Voltage (kV)":"Voltage", "Distance (cm)":"Distance"}
            
print(data_frame.head())

data_frame = data_frame.rename(columns = feature_rename_dictionary)

# different_bead_values_count = data_frame["Beads"].value_counts(dropna = False)
# print(different_bead_values_count)

data_frame = data_frame.dropna()

combine_yes_no = {"Yes" : "YES", "No" : "NO"}
data_frame["Beads"] = data_frame["Beads"].replace(combine_yes_no)

encode_yes_no = {"YES" : 1, "NO" : 0}
data_frame["Beads"] = data_frame["Beads"].replace(encode_yes_no)

# print(data_frame)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# print(data_frame.columns)

columns_to_scale = ['Chi', 'Conc.', 'delta', 'RED ', 'Voltage', 'Distance']
scaled_data = scaler.fit_transform(data_frame[columns_to_scale])
data_frame_scaled = pd.DataFrame(scaled_data, columns=columns_to_scale)
print(data_frame_scaled)


X = data_frame_scaled
y = data_frame['Beads']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)









   Flory-Huggins χ parameter = (Mvol*Ra^2)/4RT  Polymer Concentration (wt%)  \
0                                       0.0049                         22.0   
1                                       0.0049                         22.0   
2                                       0.0049                         22.0   
3                                       0.0049                         22.0   
4                                       0.0049                         22.0   

   δ* =Sum of Products of Volume Fraction and δ of Individual Solvents  \
0                                           22.77125                     
1                                           22.77125                     
2                                           22.77125                     
3                                           22.77125                     
4                                           22.77125                     

     RED  Beads  Voltage (kV)  Distance (cm)  
0  0.0751   YES          12.0    

<ipython-input-26-fc6fbef357a9>:151: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_c = X_train[y_train == c]


IndexingError: ignored

In [ ]:
# Logistic regression classifier 

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



log_reg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0,  solver='liblinear',
                                              max_iter=1000 )
log_reg.fit(X_train, y_train)


y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Logistic Regression:", accuracy)







#Supper Vector Machine classifier with RBF kernel

from sklearn.svm import SVC
svm = SVC( C=1.0, kernel='rbf', degree=3)


svm.fit(X_train, y_train)


y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of SVM", accuracy)





#Naive Gaussian Bayes classifier

from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Gaussian Bayes:", accuracy)




#Categorical Naive Bayes classifier 


from sklearn.naive_bayes import CategoricalNB

gnb_cat = CategoricalNB()
gnb_cat.fit(X_train, y_train)

y_pred = gnb_cat.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Categorical Gaussian Bayes:", accuracy)




#XGboost classifier with 200 rounds

import xgboost as xgb


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'objective': 'binary:logistic',  # Binary classification problem
    'eval_metric': 'logloss'  # Evaluation metric
}

num_rounds = 200  
model = xgb.train(params, dtrain, num_rounds)


y_pred = model.predict(dtest)
y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]



accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy of XGboost:", accuracy)


In [ ]:
#Gaussian Bayes classifier with actual covariance matrix

import numpy as np
encode_yes_no = {"YES" : 1, "NO" : 2}
data_frame["Beads"] = data_frame["Beads"].replace(encode_yes_no)
X = data_frame_scaled
y = data_frame['Beads']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

class GaussianBayes:
    # input the variant you want to use with the declaration of class
    def __init__(self, variant="different"):
        self.variant = variant

    def train(self, X_train, y_train):
        GaussianBayes.mean = {}
        std = 0
        GaussianBayes.cov = {}
        GaussianBayes.priors = {}
        #iterate over each unique class
        for c in np.unique(y_train):
            X_c = X_train[y_train == c]
            GaussianBayes.mean[c] = np.mean(X_c, axis=0)
            GaussianBayes.priors[c] = np.size(X_c)/np.size(X_train)
            std += np.std(X_c, axis=0)

        GaussianBayes.std_mean = np.mean(np.array(std))

        if(self.variant == "same"):
            GaussianBayes.cov = np.cov(X_train.T)

        elif(self.variant == "different"):
            for c in np.unique(y_train):
                X_c = X_train[y_train == c]
                GaussianBayes.cov[c] = np.ma.cov(np.array(X_c).T)

        GaussianBayes.uniques = np.unique(y_train)

    def test(self, X_test, y_test):

        pred_class=self.predict(X_test)

        pred_class = np.array(pred_class)
        true_class = np.array(y_test)

        correct_pred = 0
        for i in range(len(pred_class)):
            if pred_class[i] == true_class[i]:
                correct_pred += 1
        return correct_pred/len(pred_class)

    def predict(self, X_test):
        pred_class = []
        for sample in np.array(X_test):
            values = []

            if(self.variant == "different"):

                for c in GaussianBayes.uniques:

                    x = (sample-GaussianBayes.mean[c])
                    cov_inv = np.linalg.inv(GaussianBayes.cov[c])

                    x_new = -0.5 *(x) @ cov_inv @ (x).T

                    det = np.linalg.det(GaussianBayes.cov[c])

                    value = x_new - 0.5 * np.log(det) + np.log(GaussianBayes.priors[c])
                    values.append(value)

                pred=np.argmax(np.array(values))+1

                pred_class.append(pred)

            if(self.variant == "same"):

                for c in GaussianBayes.uniques:

                    x = (sample-GaussianBayes.mean[c])
                    cov_inv = np.linalg.inv(GaussianBayes.cov)

                    x_new = -0.5 *(x).T @ cov_inv @ (x).T
                    value = x_new

                    values.append(value)
                pred=np.argmax(np.array(values))+1
                # print(np.argmax(values))
                pred_class.append(pred)

            if(self.variant == "independent"):

                for c in GaussianBayes.uniques:
                    x = (sample-GaussianBayes.mean[c])
                    # cov_inv=np.linalg.inv(GaussianBayes.cov)
                    
                    x_new = -0.5 *(x) @ (x).T
                    value = x_new/GaussianBayes.std_mean
                    values.append(value)
                    # print(value)
                # values=np.array(values + "*")
                pred=np.argmax(np.array(values))+1
                # print(np.argmax(values))
                pred_class.append(pred)

        return np.array(pred_class)


gb = GaussianBayes(variant = "different")

gb.train(X_train, y_train)
accuracy = gb.test(X_test,y_test)
print("Accuracy of Gaussian Bayes with full covariance matrix:", accuracy)